<a href="https://colab.research.google.com/github/Podushka1337/homework/blob/main/OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ООП. Халитова Елена, 09-313, вариант 20

In [3]:
from abc import ABC, abstractmethod
from typing import List

# Исключения
class RentalServiceError(Exception):
    """Базовое исключение для сервиса проката велосипедов."""
    pass

class BikeNotFoundError(RentalServiceError):
    """Исключение при отсутствии велосипеда."""
    pass

class InvalidBikeDataError(RentalServiceError):
    """Исключение при некорректных данных велосипеда."""
    pass

# Миксины
class LoggingMixin:
    def log(self, message: str):
        print(f"[LOG] {message}")

class NotificationMixin:
    def notify(self, message: str):
        print(f"[NOTIFICATION] {message}")

# Абстрактный класс Bike
class Bike(ABC):
    def __init__(self, bike_id: int, model: str, rental_duration: int, hourly_rate: float, is_available: bool = True):
        if not model:
            raise InvalidBikeDataError("Модель не может быть пустой.")
        if rental_duration <= 0:
            raise InvalidBikeDataError("Продолжительность аренды должна быть положительным числом.")
        if hourly_rate <= 0:
            raise InvalidBikeDataError("Ставка аренды должна быть положительным числом.")
        self.__bike_id = bike_id
        self.__model = model
        self.__rental_duration = rental_duration
        self.__hourly_rate = hourly_rate
        self.__is_available = is_available

    @abstractmethod
    def calculate_rental_cost(self) -> float:
        """Вычислить стоимость аренды."""
        pass

    # Геттеры и сеттеры
    def get_id(self) -> int:
        return self.__bike_id

    def get_model(self) -> str:
        return self.__model

    def set_model(self, model: str):
        if not model:
            raise InvalidBikeDataError("Модель не может быть пустой.")
        self.__model = model

    def get_rental_duration(self) -> int:
        return self.__rental_duration

    def set_rental_duration(self, duration: int):
        if duration <= 0:
            raise InvalidBikeDataError("Продолжительность аренды должна быть положительным числом.")
        self.__rental_duration = duration

    def get_hourly_rate(self) -> float:
        return self.__hourly_rate

    def set_hourly_rate(self, rate: float):
        if rate <= 0:
            raise InvalidBikeDataError("Ставка аренды должна быть положительным числом.")
        self.__hourly_rate = rate

    def is_available(self) -> bool:
        return self.__is_available

    def set_availability(self, available: bool):
        self.__is_available = available

    def __str__(self):
        return (f"Велосипед (ID {self.get_id()}): {self.get_model()}, "
                f"длительность аренды {self.get_rental_duration()} ч, ставка {self.get_hourly_rate()} руб/ч")

    def __lt__(self, other):
        if not isinstance(other, Bike):
            return NotImplemented
        if self.get_hourly_rate() == other.get_hourly_rate():
            return self.get_rental_duration() < other.get_rental_duration()
        return self.get_hourly_rate() < other.get_hourly_rate()

    def __gt__(self, other):
        if not isinstance(other, Bike):
            return NotImplemented
        if self.get_hourly_rate() == other.get_hourly_rate():
            return self.get_rental_duration() > other.get_rental_duration()
        return self.get_hourly_rate() > other.get_hourly_rate()

# Конкретные классы велосипедов
class StandardBike(Bike):
    def calculate_rental_cost(self) -> float:
        return self.get_hourly_rate() * self.get_rental_duration()

class ElectricBike(Bike):
    def __init__(self, bike_id: int, model: str, rental_duration: int, hourly_rate: float, battery_charge_fee: float, is_available: bool = True):
        super().__init__(bike_id, model, rental_duration, hourly_rate, is_available)
        if battery_charge_fee < 0:
            raise InvalidBikeDataError("Сбор за заряд батареи не может быть отрицательным.")
        self.__battery_charge_fee = battery_charge_fee

    def calculate_rental_cost(self) -> float:
        return self.get_hourly_rate() * self.get_rental_duration() + self.__battery_charge_fee


    def __str__(self):
        base = super().__str__()
        return f"{base}, сбор за заряд: {self.__battery_charge_fee} руб"

# Сервис управления прокатом велосипедов
class BikeRentalService(LoggingMixin, NotificationMixin):
    def __init__(self):
        self._bikes: List[Bike] = []

    def add_bike(self, bike: Bike):
        if not isinstance(bike, Bike):
            raise InvalidBikeDataError("Можно добавлять только объекты Bike.")
        self._bikes.append(bike)
        self.log(f"Добавлен велосипед: {bike}")
        self.notify(f"Новый велосипед: {bike}")

    def edit_bike(self, bike_id: int, **kwargs):
        bike = self._find_by_id(bike_id)
        if 'model' in kwargs:
            bike.set_model(kwargs['model'])
        if 'rental_duration' in kwargs:
            bike.set_rental_duration(kwargs['rental_duration'])
        if 'hourly_rate' in kwargs:
            bike.set_hourly_rate(kwargs['hourly_rate'])
        if 'is_available' in kwargs:
            bike.set_availability(kwargs['is_available'])
        self.log(f"Изменен велосипед ID {bike_id}")
        self.notify(f"Велосипед обновлён: {bike}")

    def delete_bike(self, bike_id: int):
        bike = self._find_by_id(bike_id)
        self._bikes.remove(bike)
        self.log(f"Удалён велосипед: {bike}")
        self.notify(f"Велосипед удалён: {bike}")

    def get_all_bikes(self) -> List[Bike]:
        return list(self._bikes)

    def search_by_model(self, model: str) -> List[Bike]:
        return [b for b in self._bikes if model.lower() in b.get_model().lower()]

    def analyze_data(self):
        total = len(self._bikes)
        available = sum(1 for b in self._bikes if b.is_available())
        avg_cost = (sum(b.calculate_rental_cost() for b in self._bikes) / total
                    if total > 0 else 0)
        return {
            'total_bikes': total,
            'available_bikes': available,
            'average_rental_cost': avg_cost
        }

    def _find_by_id(self, bike_id: int) -> Bike:
        for b in self._bikes:
            if b.get_id() == bike_id:
                return b
        raise BikeNotFoundError(f"Велосипед с ID {bike_id} не найден.")

#
if __name__ == "__main__":
    service = BikeRentalService()

    # Добавляем велосипеды
    sb = StandardBike(1, "Giant", 3, 50.0)
    eb = ElectricBike(2, "E-BikeX", 5, 80.0, battery_charge_fee=15.0)

    service.add_bike(sb)
    service.add_bike(eb)

    # Редактируем
    service.edit_bike(1, rental_duration=4, hourly_rate=55.0)

    # Поиск по модели
    results = service.search_by_model("e-bi")
    print("\nПоиск по 'e-bi':")
    for bike in results:
        print(bike, "- Стоимость аренды:", bike.calculate_rental_cost())

    # Анализ данных
    stats = service.analyze_data()
    print("\nСтатистика по прокату велосипедов:", stats)


[LOG] Добавлен велосипед: Велосипед (ID 1): Giant, длительность аренды 3 ч, ставка 50.0 руб/ч
[NOTIFICATION] Новый велосипед: Велосипед (ID 1): Giant, длительность аренды 3 ч, ставка 50.0 руб/ч
[LOG] Добавлен велосипед: Велосипед (ID 2): E-BikeX, длительность аренды 5 ч, ставка 80.0 руб/ч, сбор за заряд: 15.0 руб
[NOTIFICATION] Новый велосипед: Велосипед (ID 2): E-BikeX, длительность аренды 5 ч, ставка 80.0 руб/ч, сбор за заряд: 15.0 руб
[LOG] Изменен велосипед ID 1
[NOTIFICATION] Велосипед обновлён: Велосипед (ID 1): Giant, длительность аренды 4 ч, ставка 55.0 руб/ч

Поиск по 'e-bi':
Велосипед (ID 2): E-BikeX, длительность аренды 5 ч, ставка 80.0 руб/ч, сбор за заряд: 15.0 руб - Стоимость аренды: 415.0

Статистика по прокату велосипедов: {'total_bikes': 2, 'available_bikes': 2, 'average_rental_cost': 317.5}
